In [2]:
!pip install mysql-connector-python


import numpy as np
import pandas as pd 
import mysql.connector


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.12
    Uninstalling protobuf-4.21.12:
      Successfully uninstalled protobuf-4.21.12


In [8]:
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'Janu@2020',
    database = 'restaurants'
)

In [9]:
data1 = pd.read_csv('orders.csv')

In [54]:
data1.shape

(10000, 6)

In [10]:
data2 = pd.read_json('users.json')

In [13]:
data3 = pd.read_sql("SELECT * FROM restaurants", conn )
data3.head()

C:\Users\singh\AppData\Local\Temp\ipykernel_23244\940895771.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data3 = pd.read_sql("SELECT * FROM restaurants", conn )


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [16]:
merge1 = pd.merge(
    data1,
    data2,
    on='user_id',
    how='left'
)


In [18]:
merge2 = pd.merge(
    merge1,
    data3,
    on='restaurant_id',
    how='left'
)


In [21]:
merge2.to_csv("final_food_delivery_dataset.csv", index=False)

In [23]:
df = pd.read_csv("final_food_delivery_dataset.csv")

In [24]:
df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [46]:
df.isnull().sum()

order_id             0
user_id              0
restaurant_id        0
order_date           0
total_amount         0
restaurant_name_x    0
name                 0
city                 0
membership           0
restaurant_name_y    0
cuisine              0
rating               0
rating_range         0
quarter              0
dtype: int64

In [ ]:
# Analysis based on question asked in the hackathon 

In [30]:
# 1. Which city has the highest total revenue from Gold members?
q1 = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

print("1. City with highest Gold-member revenue:", q1)

1. City with highest Gold-member revenue: Chennai


In [31]:
# 2. Which cuisine has the highest average order value?
q2 = (
    df.groupby("cuisine")["total_amount"]
    .mean()
    .idxmax()
)

print("2. Cuisine with highest average order value:", q2)

2. Cuisine with highest average order value: Mexican


In [32]:
# 3.  How many distinct users spent more than ₹1000 in total?
q3 = (
    df.groupby("user_id")["total_amount"]
    .sum()
    .gt(1000)
    .sum()
)

print("3. Users spending more than ₹1000:", q3)

3. Users spending more than ₹1000: 2544


In [41]:
# 4. Which restaurant rating range generated the highest total revenue?
df["rating_range"] = pd.cut(
    df["rating"],
    bins=[0, 3.5, 4, 4.5, 5],
    labels=[
        "Low (0–3.5)",
        "Low-Medium (3.5–4)",
        "Medium (4–4.5)",
        "High (4.5–5)"
    ],
    include_lowest=True
)

q4 = (
    df.groupby("rating_range")["total_amount"]
    .sum()
    .idxmax()
)

print("4. Rating range with highest revenue:", q4)


4. Rating range with highest revenue: High (4.5–5)


C:\Users\singh\AppData\Local\Temp\ipykernel_23244\3619941722.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"]


In [34]:
# 5. Among Gold members, city with highest average order value
q5 = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .idxmax()
)

print("5. City with highest AOV among Gold members:", q5)


5. City with highest AOV among Gold members: Chennai


In [35]:
# 6. Cuisine with fewest restaurants but high revenue
restaurant_count = df.groupby("cuisine")["restaurant_id"].nunique()
revenue = df.groupby("cuisine")["total_amount"].sum()

q6 = (
    pd.DataFrame({
        "restaurants": restaurant_count,
        "revenue": revenue
    })
    .sort_values(["restaurants", "revenue"], ascending=[True, False])
    .head(1)
)

print("6. Cuisine with few restaurants but high revenue:")
print(q6)


6. Cuisine with few restaurants but high revenue:
         restaurants     revenue
cuisine                         
Chinese          120  1930504.65


In [36]:
# 7. Percentage of orders placed by Gold members
q7 = round(
    (df[df["membership"] == "Gold"].shape[0] / df.shape[0]) * 100
)

print("7. % of orders by Gold members:", q7, "%")


7. % of orders by Gold members: 50 %


In [45]:
# 8. Restaurant with highest AOV but < 20 orders
restaurant_name = q8.iloc[0]["restaurant_name_x"]
aov = q8.iloc[0]["avg_order_value"]
orders = q8.iloc[0]["order_count"]

print(
    f"8. Restaurant with highest AOV but < 20 orders: "
    f"{restaurant_name} (AOV ₹{aov:.2f}, Orders {orders})"
)


8. Restaurant with highest AOV but < 20 orders: Hotel Dhaba Multicuisine (AOV ₹1040.22, Orders 13)


In [47]:
# 9. City + Cuisine combination with highest revenue 
q9 = (
    df.groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .idxmax()
)

print("9. City–Cuisine combo with highest revenue:", q9)


9. City–Cuisine combo with highest revenue: ('Regular', 'Mexican')


In [39]:
# 10. 10. Quarter with highest total revenue
df["order_date"] = pd.to_datetime(df["order_date"])
df["quarter"] = df["order_date"].dt.to_period("Q")

q10 = (
    df.groupby("quarter")["total_amount"]
    .sum()
    .idxmax()
)

print("10. Quarter with highest revenue:", q10)


10. Quarter with highest revenue: 2023Q3


C:\Users\singh\AppData\Local\Temp\ipykernel_23244\3637211820.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


In [48]:
# 11. How many orders were placed in the top revenue city among Gold members only?
gold_orders = df[df["membership"] == "Gold"].shape[0]

print(f"1. Total orders by Gold members: {gold_orders}")
print("Insight: This shows how actively Gold users place orders compared to others.\n")


1. Total orders by Gold members: 4987
Insight: This shows how actively Gold users place orders compared to others.



In [49]:
# 12 Total revenue from Hyderabad (rounded)
hyderabad_revenue = round(
    df[df["city"] == "Hyderabad"]["total_amount"].sum()
)

print(f"2. Total revenue from Hyderabad: ₹{hyderabad_revenue}")
print("Insight: Indicates Hyderabad’s contribution to overall platform revenue.\n")


2. Total revenue from Hyderabad: ₹1889367
Insight: Indicates Hyderabad’s contribution to overall platform revenue.



In [50]:
#13. How many distinct users placed at least one order?
distinct_users = df["user_id"].nunique()

print(f"3. Distinct users who placed orders: {distinct_users}")
print("Insight: Represents the active customer base of the platform.\n")


3. Distinct users who placed orders: 2883
Insight: Represents the active customer base of the platform.



In [51]:
#14. Average order value for Gold members (rounded to 2 decimals)
gold_aov = round(
    df[df["membership"] == "Gold"]["total_amount"].mean(), 2
)

print(f"4. Average order value for Gold members: ₹{gold_aov}")
print("Insight: Gold members usually have higher spending per order.\n")


4. Average order value for Gold members: ₹797.15
Insight: Gold members usually have higher spending per order.



In [52]:
#15. Orders placed for restaurants with rating ≥ 4.5
high_rating_orders = df[df["rating"] >= 4.5].shape[0]

print(f"5. Orders placed for restaurants with rating ≥ 4.5: {high_rating_orders}")
print("Insight: Shows customer preference for highly rated restaurants.\n")


5. Orders placed for restaurants with rating ≥ 4.5: 3374
Insight: Shows customer preference for highly rated restaurants.



In [53]:
#16. Orders placed in the top revenue city among Gold members
top_gold_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

orders_top_city = (
    df[(df["membership"] == "Gold") & (df["city"] == top_gold_city)]
    .shape[0]
)

print(f"6. Orders in top revenue city among Gold members ({top_gold_city}): {orders_top_city}")
print("Insight: This city drives the most Gold-member revenue and engagement.\n")


6. Orders in top revenue city among Gold members (Chennai): 1337
Insight: This city drives the most Gold-member revenue and engagement.

